In [2]:
import numpy as np
import pandas as pd
import subprocess
import glob

In [1]:
# Grab eQTL files
df = pd.read_csv("eQTL_data/tabix_ftp_paths.tsv",sep="\t")
df = df.loc[(df.tissue_label.isin(tissues)) & (df.quant_method == "ge")]
# for i, j in df.iterrows():
#     subprocess.run(f'curl {j.ftp_path} -O', shell=True)

In [ ]:
# GWAS cleaning
df = pd.read_csv("GWAS_data/GWAS_SA.tsv.gz",sep="\t")
df = df.drop(columns=["effect_allele_frequency", "p_value"])
df["variant_id"] = "chr" + df.chromosome.astype("string") + "_" + df.base_pair_location.astype("string") + "_" + df.other_allele.str.upper() + "_" + df.effect_allele.str.upper()

df = df.rename(columns={"standard_error": "SE"})
df[["variant_id", "beta", "SE"]].to_csv("GWAS_data/GWAS_cleaned.tsv.gz", sep="\t", index=False)

In [3]:
# GWAS DROPNA
gwas_df = pd.read_csv("/scratch/network/cs9095/cs551/GWAS_data/GWAS_SA.tsv.gz", sep="\t")
gwas_df = gwas_df.dropna()
gwas_df = gwas_df[["variant_id", "beta", "standard_error", "p_value"]]
gwas_df.columns = ["rsid", "beta_gwas", "se_gwas", "pval_gwas"]
gwas_df.to_csv("GWAS_data/GWAS_SA_dropna.tsv.gz", index=False, sep="\t")

In [ ]:
gwas_df = pd.read_csv("/scratch/network/cs9095/cs551/GWAS_data/GWAS_SA.tsv.gz", sep="\t")

In [ ]:
# ENSEMBL codes for genes of interest
df_sa = pd.read_csv("GWAS_data/GWAS_SA_assoc.tsv",sep="\t")
df_tw = pd.read_csv("GWAS_data/GWAS_TW_assoc.tsv",sep="\t")
df_cn = pd.read_csv("GWAS_data/GWAS_CN_assoc.tsv",sep="\t")

ensembl_genes = []
for df in [df_sa, df_tw, df_cn]:
    gene_ids = df.SNP_GENE_IDS.dropna().str.rsplit(",", expand=True)
    if len(gene_ids.columns) > 1:
        gene_ids = gene_ids[0].tolist() + gene_ids[1].dropna().str.strip().tolist()
        ensembl_genes += np.unique(df.UPSTREAM_GENE_ID.dropna().tolist() + df.DOWNSTREAM_GENE_ID.dropna().tolist() + gene_ids).tolist()
    else:
        ensembl_genes += np.unique(df.UPSTREAM_GENE_ID.dropna().tolist() + df.DOWNSTREAM_GENE_ID.dropna().tolist() + gene_ids[0].tolist()).tolist()

ensembl_genes = np.unique(ensembl_genes)

pd.DataFrame(ensembl_genes, columns=["ENSEMBL"]).to_csv("GWAS_data/ENSEMBL_genes.tsv", sep="\t", index=False)